In [1]:
!pip install -q gradio
!pip install huggingsound
!pip install unidecode

In [2]:
# cargamos el modelo de huggingsound
import re
from huggingsound import SpeechRecognitionModel

# creación del modelo
model = SpeechRecognitionModel("patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm")

05/02/2022 05:36:47 - INFO - huggingsound.speech_recognition.model - Loading model...


/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [3]:
import gradio as gr


def greet(name):
    return "Hello " + name + "!!"


demo = gr.Interface(fn=greet, inputs="text", outputs="text")

if __name__ == "__main__":
    demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://59262.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


In [4]:
import gradio as gr

bloque = ''
codigo = None
indentacion = 0
# Esto para dar seguimiento al eliminado de una linea
linea_codigo = 0

In [5]:
def sim_jac(s1, s2):

    bigrams_s1 = []
    bigrams_s2 = []

    for i in range(len(s1) - 1):
        bigrams_s1.append(s1[i:i+2])
    
    for i in range(len(s2) - 1):
        bigrams_s2.append(s2[i:i+2])

    c_common = 0

    for i in bigrams_s1:
        if bigrams_s2.count(i) > 0:
            c_common += 1

    return c_common / ((len(s1) - 1) + (len(s2) - 1) - c_common)

## Funciones por implementar


In [6]:
def crear_clase():
    global indentacion
    # ....
    
    indentacion += 1
    pass         

In [7]:
def crear_funcion(instruccion):
    """
    Crea el template de la estructura de una función

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado
    recomendacion: str
        Una sugerencia o fallo
    """

    global indentacion

    # guarda los avisos o recomendaciones que el programa te hace
    recomendacion = ''

    # guarda la línea de código
    output = ''
    
    # pivote que ayuda a definir el nombre de una función
    before_keyword, keyword, after_keyword = instruccion.partition('nombre')

    # verifica que haya o esté escrita la frase "nombre"
    if len(after_keyword) == 0:
        recomendacion = f'¡No me dijiste el nombre de la función!'

    # de otro modo, si tiene nombre la función
    else:

        # obtenemos el nombre de la función por el usuario
        name_func = after_keyword.split(' ')[1]

        # verificamos si no desea poner parametros                                
        if instruccion.strip().split(' ')[-1] == name_func:
            parametros = ''

        # de otro modo, si desea una función con parámetros
        else:
            before_keyword, keyword, after_keyword = instruccion.partition('parametros')

            # verifica que si exista el nombre de los parámetros
            if len(after_keyword) == 0:
                parametros = ''
                recomendacion = f'¡No me dijiste el nombre de los parámetros!'

            # escribe como parámetros todo lo que está después de "parámetros"
            else:
                parametros = after_keyword.split(' ')[1:]
                parametros = ', '.join(parametros)

        # indenta aunque marque que detecte que no le dije parámetros
        if not recomendacion or recomendacion == '¡No me dijiste el nombre de los parámetros!':
            indentacion += 1

        # concatenación del nombre y parámetros de la función
        output = f'def {name_func}({parametros}):\n' + '\t' * indentacion
    return output, recomendacion

In [8]:
crear_funcion('definir función con nombre magica y parametros x y z')[0]

'def magica(x, y, z):\n\t'

In [9]:
diccionario_fonetico={'alfa':'a','india':'i'}


dict_numeros = {
    'uno': 1,
    'dos': 2,
    'tres': 3,
    'cuatro':4,
    'cinco': 5,
    'seis': 6,
    'siete': 7,
    'ocho': 8,
    'nueve': 9,
    'diez': 10,
    'once': 11,
    'doce': 12,
    'trece': 13,
    'catorce': 14,
    'quince': 15,
    'dieciseis': 16,
    'diecisiete': 17,
    'dieciocho': 18,
    'diecinueve': 19,
    'veinte': 20,
    'treinta': 30,
    'cuarenta': 40,
    'cicuenta': 50,
}

def numero(text):
    """Convierte un texto de numero en numero entero (int)"""
    global dict_numeros    
    return dict_numeros[text] 
    
# Diccionario de funciones
tipos_datos ={
    'numero': numero
}

In [25]:
def encontrar_palabras(transcript,cjto_palabras):
    '''
    Toma un string (en minúsculos) y un conjunto de palabras. Busca el primer match
    de cjto_palabras en transcript y particiona el string en:
        1. El slice de la cadena antes del primer match (antes_palabra)
        2. La cadena del primer match (coincidencia de cjto_palabras)
        3. El slice de la cadena después del match (despues_palabra)
    '''
    inicio,final=list(re.finditer(r'|'.join(cjto_palabras),transcript))[0].span()
    antes_palabra=transcript[:inicio].strip()
    despues_palabra=transcript[final:].strip()
    palabra=transcript[inicio:final]
    return antes_palabra,palabra,despues_palabra

def crear_condicional(transcript):
    ''' 
    Toma transcript y condicional (if,elif,while) y devuelve código
    El transcript ya está preprocesado para comenzar con la palabra "mientras"

    Test
    >>> mientras variable india sea menor igual a numero seis => while (i<=6):
    '''    
    global bloque

    keyword_mapeo={'mientras':'while','si':'if','no si':'elif'}
    antes_keyword,keyword,desp_keyword=encontrar_palabras(transcript,keyword_mapeo.keys())
    cadena=keyword_mapeo[keyword]
    bloque = keyword

    # Primera división
    condicional_mapeo={'menor estricto':'<','menor o igual':'<=','igual':'==','diferente':'!='
    ,'mayor estricto':'>','mayor o igual':'>='}
    cjto_condicional=condicional_mapeo.keys()
    antes_condicional,palabra_condicional,despues_condicional=encontrar_palabras(transcript,cjto_condicional) 
    
    
    # Buscar en antes condicional la lista de variables
    

    a_var,var,d_var=encontrar_palabras(antes_condicional,['variable'])
    nombre_var=d_var.split(' ')[0]

    if diccionario_fonetico.get(nombre_var,False):
        nombre_var=diccionario_fonetico[nombre_var]


    cadena+=' '+nombre_var+' ' +condicional_mapeo[palabra_condicional]

    # Buscar en despues_condicional el número 

    valor=despues_condicional.split(' ')[-1]

    if dict_numeros.get(valor,False):
        valor=str(dict_numeros[valor])

    global indentacion
    indentacion+=1

    return cadena+' '+ valor+':\n'+ '\t'* indentacion+'|', ''

indentacion=0
a=['si variable india es menor o igual a numero seis','mientras variable alpha es menor igual a numero dos']
for ai in a:
    print(crear_condicional(ai))


('if i <= 6:\n\t|', '')
('while alpha == 2:\n\t\t|', '')


In [50]:
   
def crear_variable(instruccion):
    """
    Estructura:
    definir variable con nombre [nombre_variable] igual a /*objeto_basico* valor/ 

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado
    recomendacion: str
        Una sugerencia o fallo

    Testing
    -------
    >>> definir variable con nombre india igual a número uno
    >>> Definir variable con nombre i igual a numero 1 (int)
    >>> Definir variable con nombre i igual a numero 3.5 (float) 
    >>> Definir variable con nombre i igual a cadena hola (string) 
    >>> Definir variable con nombre i igual a lista/dic (string)   
    """     
    global indentacion

    # pivote que ayuda a definir el nombre de la variable
    before_keyword, keyword, after_keyword = instruccion.partition('nombre')
    after_keyword_list = after_keyword.strip().split(' ')
    name_variable = after_keyword_list[0]
    # Como sabemos que despues del nombre va seguido de "igual a"
    tipo_dato = after_keyword_list[3]        
    #print(after_keyword_list[4:]) -> lista
    valor = tipos_datos[tipo_dato](after_keyword_list[4:][0])        
    # Verificamos si es una palabra fonetica
    if diccionario_fonetico.get(name_variable,False):
        name_variable=diccionario_fonetico[name_variable] 

    codigo_generado = f'{name_variable} = {valor}\n'+ '\t' * indentacion + '|'
    return codigo_generado, ''    

### Test de la funcion crear variable

In [12]:
crear_variable('definir variable con nombre india igual a numero cuatro')

'i = 4\n\t\t'

In [13]:
import unidecode
import re

def main(instruccion):
    global bloque

    # Obtenemos el bloque que será del contexto
    # Porque son los unicos que va modificar la identacion
    # bloque = {funcion, clase, estructura_control}        
    bloques_patrones = r'(funcion|clase|mientras|si)'
    bloque = re.findall(bloques_patrones, instruccion)[0]

    # lista de comandos disponibles
    # []: Todo lo que esta corchetes son los valores que vamos obtener
    # (): Son sentencias opciones para agregar más cosas    
    comandos = [
                'funcion',  # con nombre [nameFuc](y parámetros [x y z])                
                'variable', # con nombre [nombre_variable] igual a /objeto_basico/
                'mientras',
                'cadena',
                'numero',
                'lista',
                'diccionario',                                                   
                ]

    # Lista de funciones que nos devolveran las plantillas
    plantillas = [crear_funcion, crear_variable]

    # buscar el comando correspondiente más cercano al deseado
    scores = []
    ind_max_score = 0

    # similitud de Jaccard para las primeras dos palabras de la grabación
    for i, comando in enumerate(comandos):
        scores.append(sim_jac(instruccion.split(' ')[1], comando))

        # guardamos índice de la instrucción con más similitud
        if scores[-1] > scores[ind_max_score]:
            ind_max_score = i
    
    return plantillas[ind_max_score](instruccion)

## Test funcionalidad de la aplicación

### Probando frase 1

In [16]:
# Probando cuando se desea crear una funcion
#codigo_generado, sugerencia = main('Definir funcion con nombre saludar y parametros mensaje')
print('Codigo generado')
print(codigo_generado)
print('---------------')
print('Sugerencia')
print(sugerencia)

Codigo generado
def saludar(mensaje):
			
---------------
Sugerencia



El valor de la variable de `codigo_generado` es que se mandara a la interfaz del usuario.

### Probando frase 2

In [17]:
# Probando cuando se desea crear una funcion
voice1 = 'definir variable con nombre india igual a número uno'
#codigo_generado, sugerencia = main(voice1)
print('Codigo generado')
print(codigo_generado)
print('---------------')
print('Sugerencia')
print(sugerencia)

Codigo generado
def saludar(mensaje):
			
---------------
Sugerencia



In [18]:
import unidecode
import re

def main2(instruccion):
    global bloque
    
    candidatos_instruccion = instruccion.split(' ')[:5]

    # lista de comandos disponibles
    # []: Todo lo que esta corchetes son los valores que vamos obtener
    comandos = [
                'funcion',  # con nombre [nameFuc](y parámetros [x y z])
                'mientras',
                'variable', # con nombre [nombre_variable] igual a /objeto_basico/
                'cadena',
                'numero',
                'lista',
                'diccionario'                       
                ]

    # Lista de funciones que nos devolveran las plantillas
    plantillas = [crear_funcion, crear_condicional, crear_variable]

    # buscar el comando correspondiente más cercano al deseado
    scores = [[] for _ in range(len(comandos))]

    ind_max_score = [0, 0]

    # similitud de Jaccard para las primeras dos palabras de la grabación
    
    for i, comando in enumerate(comandos):
        for j, palabra in enumerate(candidatos_instruccion):
            scores[i].append(sim_jac(palabra, comando))

            # guardamos índice de la instrucción con más similitud
            if scores[i][-1] > scores[ind_max_score[0]][ind_max_score[1]]:
                ind_max_score = [i, j]    
    
    return plantillas[ind_max_score[0]](instruccion)

## Prueba con similación

Voz 1

In [ ]:
main2('mientras variable india es menor o igual a seis')

Voz 2

In [ ]:
main2('definir variable con nombre india igual a numero cinco')

In [51]:
def agregar_adentro(codigo, transcipcion):
    codigo2, recomendacion = main2(transcipcion)
    return codigo[:-1] + codigo2, recomendacion

In [52]:
# Variables globales
bloque = '' # Define el contexto (si es función, condicional, ciclo, etc.)
codigo = None # Guarda el código hasta el momento
indentacion = 0 # Nivel de indentación
linea_codigo = 0 # Esto para dar seguimiento al eliminado de una linea

def transcribe(audio, state=""):
    global bloque
    global codigo
    

    transcriptions_es = model.transcribe([audio])[0]
    # quitamos el acento de la transcripcion
    frase = unidecode.unidecode(transcriptions_es['transcription']).lower()
    
    if not bloque:
        # Significa que es la primera vez
        codigo, recomendacion = main2(frase)        
    else:        
        if bloque == 'funcion':
            codigo, recomendacion = agregar_adentro(codigo, frase)
        elif bloque == 'clase':
            pass
        elif bloque == 'mientras':
            codigo, recomendacion = agregar_adentro(codigo, frase)
            pass
        elif bloque == 'si':
            pass
    
    
    # Prueba
    # codigo = transcriptions_es['transcription']
    # recomendacion = "hola"
    return codigo, recomendacion

inputs = gr.inputs.Audio(label="Dar click para escuchar tu voz", type="filepath", source="microphone")
output1 = gr.outputs.Textbox(label="Asi se ve tu código")
output2 = gr.outputs.Textbox(label="Sugerencia")

title = "Expresate con voz"
description = "Aplicación que ayuda a programar a traves de tu voz"
examples = ['definir función con nombre magica y parámetros x y z','definir función', 'definir variable', 'definir clase']
article = "<a  style='color:#eb9f59;' href = 'https://github.com/gandres-dev/Hackaton-Common-Voice'> Repositorio de la app"
demo = gr.Interface(fn=transcribe, inputs=inputs, outputs=[output1,output2],
                    title=title, description=description, article=article,
                    allow_flagging="never", theme="darkpeach", examples=examples,
                    # live=True
                    )

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:317: UserWarning: 'darkpeach' theme name is deprecated, using dark-peach instead.
  f"'{theme}' theme name is deprecated, using {DEPRECATED_THEME_MAP[theme]} instead."


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
Running on public URL: https://59535.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


## Probando la aplicación con widget

Crea el widget de audio

In [32]:
!pip install ipywebrtc

     |████████████████████████████████| 260 kB 5.0 MB/s 


In [35]:
from google.colab import output
output.enable_custom_widget_manager()

In [36]:
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Reescribir audio ahora en formato .wav

In [43]:
# Escribe el formato en webm
with open('test_audio_avitua_linea5.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Convierte en webm -> wav
!ffmpeg -i test_audio_avitua_linea5.webm -ac 1 -f wav test_audio_avitua_linea5.wav -y -hide_banner -loglevel panic


## Audios de Avitua

In [ ]:
# Escribe el formato en webm
with open('./grabaciones-webm/test_audio_avitua_linea4.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Convierte en webm -> wav
!ffmpeg -i ./grabaciones-webm/test_audio_avitua_linea4.webm -ac 1 -f wav ./grabaciones-wav/test_audio_avitua_linea4.wav -y -hide_banner -loglevel panic

## Audios Vizuet

In [ ]:
# Escribe el formato en webm
with open('./grabaciones-webm/test_audio_vizuet.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Convierte en webm -> wav
!ffmpeg -i ./grabaciones-webm/test_audio_vizuet.webm -ac 1 -f wav ./grabaciones-wav/test_audio_vizuet.wav -y -hide_banner -loglevel panic

## Audios Memo

In [ ]:
# Escribe el formato en webm
with open('./grabaciones-webm/test_audio_memo.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Convierte en webm -> wav
!ffmpeg -i ./grabaciones-webm/test_audio_memo.webm -ac 1 -f wav ./grabaciones-wav/test_audio_memo.wav -y -hide_banner -loglevel panic

## Modelos

In [39]:
audio_paths = ["test_audio_avitua_linea4.wav"]

# Obtenemos la frase del usuario
transcriptions_es = model.transcribe(audio_paths)[0]
# Obtener la frase
transcriptions_es['transcription']

100%|██████████| 1/1 [00:07<00:00,  7.65s/it]


'mientras variable india es menor o igual a seis'

In [41]:
bloque = '' # Define el contexto (si es función, condicional, ciclo, etc.)
codigo = None # Guarda el código hasta el momento
indentacion = 0 # Nivel de indentación
linea_codigo = 0 # Esto para dar seguimiento al eliminado de una linea

In [42]:
transcribe("test_audio_avitua_linea4.wav")

100%|██████████| 1/1 [00:07<00:00,  7.05s/it]


('while i <= 6:\n\t|', '')

## Segunda grabacion

In [44]:
transcribe("test_audio_avitua_linea5.wav")

100%|██████████| 1/1 [00:07<00:00,  7.46s/it]


('while i <= 6:\n\t|i = 5\n\t', '')

In [ ]:
voice1 = 'definir variable con nombre india igual a número uno'
voice2 = 'mientras india sea menoro igual a número seis'
voice3 = 'imprime india producto cadena guion'
voice4 = 'definir variable con nombre india igual a india más uno'

Para no usar la interfaz de usuario podemos simplementar llamar la funcion:

In [ ]:
# Variables globales
bloque = '' # Define el contexto (si es función, condicional, ciclo, etc.)
codigo = None # Guarda el código hasta el momento
indentacion = 0 # Nivel de indentación
linea_codigo = 0 # Esto para dar seguimiento al eliminado de una linea

def transcribe_test(audio):
    global bloque
    global codigo
    
    transcriptions_es = model.transcribe([audio])[0]
    # quitamos el acento de la transcripcion
    frase = unidecode.unidecode(transcriptions_es['transcription'])    
    
    if not bloque:
        # Significa que es la primera vez
        codigo, recomendacion = main(frase)        
    else:        
        if bloque == 'funcion':
            codigo, recomendacion = agregar_adentro(codigo, frase)
        elif bloque == 'clase':
            pass
    
    # codigo = 
    # recomendacion = "hola"
    return codigo, recomendacion

In [ ]:
transcribe_test('./grabaciones-wav/test_audio_memo.wav')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17740a58-e960-4de3-bb66-1575286930f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>